In [33]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer


In [34]:
text="""
Recurrent Neural Networks
Humans don’t start their thinking from scratch every second. As you read this essay, you understand each word based on your understanding of previous words. You don’t throw everything away and start thinking from scratch again. Your thoughts have persistence.

Traditional neural networks can’t do this, and it seems like a major shortcoming. For example, imagine you want to classify what kind of event is happening at every point in a movie. It’s unclear how a traditional neural network could use its reasoning about previous events in the film to inform later ones.

Recurrent neural networks address this issue. They are networks with loops in them, allowing information to persist.


Recurrent Neural Networks have loops.
In the above diagram, a chunk of neural network, A
, looks at some input xt
 and outputs a value ht
. A loop allows information to be passed from one step of the network to the next.

These loops make recurrent neural networks seem kind of mysterious. However, if you think a bit more, it turns out that they aren’t all that different than a normal neural network. A recurrent neural network can be thought of as multiple copies of the same network, each passing a message to a successor. Consider what happens if we unroll the loop:

An unrolled recurrent neural network.
An unrolled recurrent neural network.
This chain-like nature reveals that recurrent neural networks are intimately related to sequences and lists. They’re the natural architecture of neural network to use for such data.

And they certainly are used! In the last few years, there have been incredible success applying RNNs to a variety of problems: speech recognition, language modeling, translation, image captioning… The list goes on. I’ll leave discussion of the amazing feats one can achieve with RNNs to Andrej Karpathy’s excellent blog post, The Unreasonable Effectiveness of Recurrent Neural Networks. But they really are pretty amazing.

Essential to these successes is the use of “LSTMs,” a very special kind of recurrent neural network which works, for many tasks, much much better than the standard version. Almost all exciting results based on recurrent neural networks are achieved with them. It’s these LSTMs that this essay will explore.

The Problem of Long-Term Dependencies
One of the appeals of RNNs is the idea that they might be able to connect previous information to the present task, such as using previous video frames might inform the understanding of the present frame. If RNNs could do this, they’d be extremely useful. But can they? It depends.

Sometimes, we only need to look at recent information to perform the present task. For example, consider a language model trying to predict the next word based on the previous ones. If we are trying to predict the last word in “the clouds are in the sky,” we don’t need any further context – it’s pretty obvious the next word is going to be sky. In such cases, where the gap between the relevant information and the place that it’s needed is small, RNNs can learn to use the past information.


But there are also cases where we need more context. Consider trying to predict the last word in the text “I grew up in France… I speak fluent French.” Recent information suggests that the next word is probably the name of a language, but if we want to narrow down which language, we need the context of France, from further back. It’s entirely possible for the gap between the relevant information and the point where it is needed to become very large.

Unfortunately, as that gap grows, RNNs become unable to learn to connect the information.

Neural networks struggle with long term dependencies.
In theory, RNNs are absolutely capable of handling such “long-term dependencies.” A human could carefully pick parameters for them to solve toy problems of this form. Sadly, in practice, RNNs don’t seem to be able to learn them. The problem was explored in depth by Hochreiter (1991) [German] and Bengio, et al. (1994), who found some pretty fundamental reasons why it might be difficult.

Thankfully, LSTMs don’t have this problem!

LSTM Networks
Long Short Term Memory networks – usually just called “LSTMs” – are a special kind of RNN, capable of learning long-term dependencies. They were introduced by Hochreiter & Schmidhuber (1997), and were refined and popularized by many people in following work.1 They work tremendously well on a large variety of problems, and are now widely used.

LSTMs are explicitly designed to avoid the long-term dependency problem. Remembering information for long periods of time is practically their default behavior, not something they struggle to learn!

All recurrent neural networks have the form of a chain of repeating modules of neural network. In standard RNNs, this repeating module will have a very simple structure, such as a single tanh layer.


The repeating module in a standard RNN contains a single layer.
LSTMs also have this chain like structure, but the repeating module has a different structure. Instead of having a single neural network layer, there are four, interacting in a very special way.

A LSTM neural network.
The repeating module in an LSTM contains four interacting layers.
Don’t worry about the details of what’s going on. We’ll walk through the LSTM diagram step by step later. For now, let’s just try to get comfortable with the notation we’ll be using.


In the above diagram, each line carries an entire vector, from the output of one node to the inputs of others. The pink circles represent pointwise operations, like vector addition, while the yellow boxes are learned neural network layers. Lines merging denote concatenation, while a line forking denote its content being copied and the copies going to different locations.
The Core Idea Behind LSTMs
The key to LSTMs is the cell state, the horizontal line running through the top of the diagram.

The cell state is kind of like a conveyor belt. It runs straight down the entire chain, with only some minor linear interactions. It’s very easy for information to just flow along it unchanged.


The LSTM does have the ability to remove or add information to the cell state, carefully regulated by structures called gates.

Gates are a way to optionally let information through. They are composed out of a sigmoid neural net layer and a pointwise multiplication operation.


The sigmoid layer outputs numbers between zero and one, describing how much of each component should be let through. A value of zero means “let nothing through,” while a value of one means “let everything through!”

An LSTM has three of these gates, to protect and control the cell state.

Step-by-Step LSTM Walk Through
The first step in our LSTM is to decide what information we’re going to throw away from the cell state. This decision is made by a sigmoid layer called the “forget gate layer.” It looks at ht−1
 and xt
, and outputs a number between 0
 and 1
 for each number in the cell state Ct−1
. A 1
 represents “completely keep this” while a 0
 represents “completely get rid of this.”

Let’s go back to our example of a language model trying to predict the next word based on all the previous ones. In such a problem, the cell state might include the gender of the present subject, so that the correct pronouns can be used. When we see a new subject, we want to forget the gender of the old subject.


The next step is to decide what new information we’re going to store in the cell state. This has two parts. First, a sigmoid layer called the “input gate layer” decides which values we’ll update. Next, a tanh layer creates a vector of new candidate values, C~t
, that could be added to the state. In the next step, we’ll combine these two to create an update to the state.

In the example of our language model, we’d want to add the gender of the new subject to the cell state, to replace the old one we’re forgetting.


It’s now time to update the old cell state, Ct−1
, into the new cell state Ct
. The previous steps already decided what to do, we just need to actually do it.

We multiply the old state by ft
, forgetting the things we decided to forget earlier. Then we add it∗C~t
. This is the new candidate values, scaled by how much we decided to update each state value.

In the case of the language model, this is where we’d actually drop the information about the old subject’s gender and add the new information, as we decided in the previous steps.


Finally, we need to decide what we’re going to output. This output will be based on our cell state, but will be a filtered version. First, we run a sigmoid layer which decides what parts of the cell state we’re going to output. Then, we put the cell state through tanh
 (to push the values to be between −1
 and 1
) and multiply it by the output of the sigmoid gate, so that we only output the parts we decided to.

For the language model example, since it just saw a subject, it might want to output information relevant to a verb, in case that’s what is coming next. For example, it might output whether the subject is singular or plural, so that we know what form a verb should be conjugated into if that’s what follows next.


Variants on Long Short Term Memory
What I’ve described so far is a pretty normal LSTM. But not all LSTMs are the same as the above. In fact, it seems like almost every paper involving LSTMs uses a slightly different version. The differences are minor, but it’s worth mentioning some of them.

One popular LSTM variant, introduced by Gers & Schmidhuber (2000), is adding “peephole connections.” This means that we let the gate layers look at the cell state.


The above diagram adds peepholes to all the gates, but many papers will give some peepholes and not others.

Another variation is to use coupled forget and input gates. Instead of separately deciding what to forget and what we should add new information to, we make those decisions together. We only forget when we’re going to input something in its place. We only input new values to the state when we forget something older.


A slightly more dramatic variation on the LSTM is the Gated Recurrent Unit, or GRU, introduced by Cho, et al. (2014). It combines the forget and input gates into a single “update gate.” It also merges the cell state and hidden state, and makes some other changes. The resulting model is simpler than standard LSTM models, and has been growing increasingly popular.

A gated recurrent unit neural network.
These are only a few of the most notable LSTM variants. There are lots of others, like Depth Gated RNNs by Yao, et al. (2015). There’s also some completely different approach to tackling long-term dependencies, like Clockwork RNNs by Koutnik, et al. (2014).

Which of these variants is best? Do the differences matter? Greff, et al. (2015) do a nice comparison of popular variants, finding that they’re all about the same. Jozefowicz, et al. (2015) tested more than ten thousand RNN architectures, finding some that worked better than LSTMs on certain tasks.

Conclusion
Earlier, I mentioned the remarkable results people are achieving with RNNs. Essentially all of these are achieved using LSTMs. They really work a lot better for most tasks!

Written down as a set of equations, LSTMs look pretty intimidating. Hopefully, walking through them step by step in this essay has made them a bit more approachable.

LSTMs were a big step in what we can accomplish with RNNs. It’s natural to wonder: is there another big step? A common opinion among researchers is: “Yes! There is a next step and it’s attention!” The idea is to let every step of an RNN pick information to look at from some larger collection of information. For example, if you are using an RNN to create a caption describing an image, it might pick a part of the image to look at for every word it outputs. In fact, Xu, et al. (2015) do exactly this – it might be a fun starting point if you want to explore attention! There’s been a number of really exciting results using attention, and it seems like a lot more are around the corner…

Attention isn’t the only exciting thread in RNN research. For example, Grid LSTMs by Kalchbrenner, et al. (2015) seem extremely promising. Work using RNNs in generative models – such as Gregor, et al. (2015), Chung, et al. (2015), or Bayer & Osendorfer (2015) – also seems very interesting. The last few years have been an exciting time for recurrent neural networks, and the coming ones promise to only be more so!

Acknowledgments
I’m grateful to a number of people for helping me better understand LSTMs, commenting on the visualizations, and providing feedback on this post.

I’m very grateful to my colleagues at Google for their helpful feedback, especially Oriol Vinyals, Greg Corrado, Jon Shlens, Luke Vilnis, and Ilya Sutskever. I’m also thankful to many other friends and colleagues for taking the time to help me, including Dario Amodei, and Jacob Steinhardt. I’m especially thankful to Kyunghyun Cho for extremely thoughtful correspondence about my diagrams.

Before this post, I practiced explaining LSTMs during two seminar series I taught on neural networks. Thanks to everyone who participated in those for their patience with me, and for their feedback.

In addition to the original authors, a lot of people contributed to the modern LSTM. A non-comprehensive list is: Felix Gers, Fred Cummins, Santiago Fernandez, Justin Bayer, Daan Wierstra, Julian Togelius, Faustino Gomez, Matteo Gagliolo, and Alex Graves.↩
"""

In [35]:
token=Tokenizer()

token.fit_on_texts([text])
token.word_index

{'the': 1,
 'to': 2,
 'a': 3,
 'of': 4,
 'and': 5,
 'in': 6,
 'is': 7,
 'we': 8,
 'neural': 9,
 'are': 10,
 'for': 11,
 'state': 12,
 'this': 13,
 'information': 14,
 'it': 15,
 'be': 16,
 'by': 17,
 'cell': 18,
 'recurrent': 19,
 'networks': 20,
 'network': 21,
 'that': 22,
 'lstms': 23,
 'on': 24,
 'what': 25,
 'step': 26,
 'rnns': 27,
 'lstm': 28,
 'they': 29,
 'next': 30,
 '”': 31,
 'it’s': 32,
 'an': 33,
 'et': 34,
 'al': 35,
 'layer': 36,
 'as': 37,
 'have': 38,
 'like': 39,
 'with': 40,
 'some': 41,
 'but': 42,
 'through': 43,
 'new': 44,
 'word': 45,
 'previous': 46,
 'example': 47,
 'at': 48,
 'one': 49,
 'these': 50,
 'if': 51,
 'all': 52,
 'language': 53,
 'long': 54,
 'term': 55,
 'might': 56,
 'only': 57,
 'going': 58,
 'output': 59,
 '2015': 60,
 'from': 61,
 'you': 62,
 'do': 63,
 'them': 64,
 'more': 65,
 'such': 66,
 'very': 67,
 'forget': 68,
 'don’t': 69,
 'each': 70,
 'want': 71,
 'can': 72,
 'there': 73,
 'using': 74,
 'need': 75,
 'model': 76,
 '–': 77,
 'also': 7

In [36]:
io=[]

for i in text.split("\n"):
    seq=token.texts_to_sequences([i])[0]
    for t in range(1,len(seq)):
        io.append(seq[:t+1])

In [97]:
len(token.word_index)

711

In [37]:
io

[[19, 9],
 [19, 9, 20],
 [308, 69],
 [308, 69, 198],
 [308, 69, 198, 84],
 [308, 69, 198, 84, 199],
 [308, 69, 198, 84, 199, 61],
 [308, 69, 198, 84, 199, 61, 200],
 [308, 69, 198, 84, 199, 61, 200, 85],
 [308, 69, 198, 84, 199, 61, 200, 85, 309],
 [308, 69, 198, 84, 199, 61, 200, 85, 309, 37],
 [308, 69, 198, 84, 199, 61, 200, 85, 309, 37, 62],
 [308, 69, 198, 84, 199, 61, 200, 85, 309, 37, 62, 310],
 [308, 69, 198, 84, 199, 61, 200, 85, 309, 37, 62, 310, 13],
 [308, 69, 198, 84, 199, 61, 200, 85, 309, 37, 62, 310, 13, 148],
 [308, 69, 198, 84, 199, 61, 200, 85, 309, 37, 62, 310, 13, 148, 62],
 [308, 69, 198, 84, 199, 61, 200, 85, 309, 37, 62, 310, 13, 148, 62, 201],
 [308, 69, 198, 84, 199, 61, 200, 85, 309, 37, 62, 310, 13, 148, 62, 201, 70],
 [308,
  69,
  198,
  84,
  199,
  61,
  200,
  85,
  309,
  37,
  62,
  310,
  13,
  148,
  62,
  201,
  70,
  45],
 [308,
  69,
  198,
  84,
  199,
  61,
  200,
  85,
  309,
  37,
  62,
  310,
  13,
  148,
  62,
  201,
  70,
  45,
  86],
 [30

In [13]:
max_len=max([len(x) for x in io])
max_len

133

In [14]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
input_squence=pad_sequences(io,maxlen=max_len,padding='pre')
input_squence

array([[  0,   0,   0, ...,   0,  19,   9],
       [  0,   0,   0, ...,  19,   9,  20],
       [  0,   0,   0, ...,   0, 308,  69],
       ...,
       [  0,   0,   0, ..., 708,   5, 709],
       [  0,   0,   0, ...,   5, 709, 710],
       [  0,   0,   0, ..., 709, 710, 711]])

In [41]:
X=input_squence[:,:-1]
y=input_squence[:,-1]
print(X[14])

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0 308  69 198  84 199  61 200
  85 309  37  62 310  13]


In [32]:
from tensorflow.keras.utils import to_categorical
y=to_categorical(y,num_classes=712)
y

MemoryError: Unable to allocate 1.87 TiB for an array with shape (360944128, 712) and data type float64

In [41]:
from tensorflow.keras.layers import Dense,LSTM,Embedding
from tensorflow.keras.models import Sequential

model=Sequential()
model.add(Embedding(712,100))
model.add(LSTM(150))
model.add(Dense(712,activation='softmax'))

In [43]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [44]:
model.fit(X,y,epochs=50)

Epoch 1/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 8s 87ms/step - accuracy: 0.0452 - loss: 6.3422
Epoch 2/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 6s 86ms/step - accuracy: 0.0611 - loss: 5.8013
Epoch 3/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 6s 85ms/step - accuracy: 0.0536 - loss: 5.7627
Epoch 4/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 6s 90ms/step - accuracy: 0.0601 - loss: 5.6192
Epoch 5/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 6s 86ms/step - accuracy: 0.0746 - loss: 5.5072
Epoch 6/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 6s 83ms/step - accuracy: 0.0982 - loss: 5.3283
Epoch 7/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 6s 87ms/step - accuracy: 0.1190 - loss: 5.1372
Epoch 8/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 6s 88ms/step - accuracy: 0.1265 - loss: 4.8763
Epoch 9/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 6s 81ms/step - accuracy: 0.1483 - loss: 4.6352
Epoch 10/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 6s 89ms/step - accuracy: 0.1685 - loss: 4.3881
Epoch 11/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 6s 83ms/step - accuracy: 0.2083 - loss: 4.1072
Epoch 12/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 6s 83ms/step - accuracy:

In [46]:
model.save("D:\python programs\All\.venv\Include\jupyter_files\Deep-Learning\LSTM\MY_Model.keras")

In [50]:
import mlflow
import mlflow.keras

with mlflow.start_run():
    mlflow.log_param("epochs", 50)
    model = tf.keras.models.load_model('D:\python programs\All\.venv\Include\jupyter_files\Deep-Learning\LSTM\MY_Model.keras')
    mlflow.keras.log_model(model, "model")

print("Model and metrics logged successfully.")

2024/06/05 00:17:37 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.


Model and metrics logged successfully.


d:\python programs\All\.venv\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [29]:
import tensorflow as tf

model = tf.keras.models.load_model('D:\python programs\All\.venv\Include\jupyter_files\Deep-Learning\LSTM\MY_Model.keras')

xtest=[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0, 308,  69, 198,  84, 199,  61, 200,  85, 309,  37,  62,
       310]

import numpy as np

xtest = np.array(xtest)
xtest = xtest.reshape(1, len(xtest), 1) 

y = model.predict(xtest)
max_index = np.argmax(y)
max_index

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step


13

In [140]:
tt="Recurrent Neural"
xt=token.texts_to_sequences([tt])[0]
xt
sq=pad_sequences([xt],maxlen=133,padding='pre')
sq[0]
n=np.array(sq[0])
n=n.reshape(1,len(sq[0]),1)
y=model.predict(n)
max_index = np.argmax(y)
max_index


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step


20

In [143]:
dict=token.word_index
dict
def get_key_from_value(d, value):
    for key, val in d.items():
        if val == value:
            return key
    return None

key=get_key_from_value(dict,20)
key
# df = pd.DataFrame(dict.items(), columns=['Word', 'Index'])
# df[df['Index']==20]

'networks'

In [93]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

class app:
    def __init__(self,word):
        self.io=[]
        self.text_token=token.texts_to_sequences([word])[0]
        self.io.append(word)
        self.input_squence=pad_sequences(self.io,maxlen=133,padding='pre')
        print(self.text_token)
        print(self.input_squence)
    def predict_next_val():
        pass
    def val_to_word():
        pass

In [94]:
ob=app("Recurrent Neural")
ob

ValueError: invalid literal for int() with base 10: 'Recurrent Neural'